# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [12]:
%help

####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Session ID: 22b17c98-80fd-46fc-8fd7-f6f44c01fa39
Applying the following default arguments:
--glue_kernel_version 1.0.2
--enable-glue-datacatalog true
Waiting for session 22b17c98-80fd-46fc-8fd7-f6f44c01fa39 to get into ready status...
Session 22b17c98-80fd-46fc-8fd7-f6f44c01fa39 has been created.



#### Example: Create a DynamicFrame from a table in the AWS Glue Data Catalog and display its schema


In [12]:
# Read from the customers table in the glue data catalog using a dynamic frame
dynamicFrameCustomers = glueContext.create_dynamic_frame.from_catalog(
database = "pyspark_tutorial_db", 
table_name = "customers"
)

# Show the top 10 rows from the dynamic dataframe
dynamicFrameCustomers.show(3)

{"customerid": 293, "firstname": "Catherine", "lastname": "Abel", "fullname": "Catherine Abel"}
{"customerid": 295, "firstname": "Kim", "lastname": "Abercrombie", "fullname": "Kim Abercrombie"}
{"customerid": 297, "firstname": "Humberto", "lastname": "Acevedo", "fullname": "Humberto Acevedo"}


#### Example: Convert the DynamicFrame to a Spark DataFrame and display a sample of the data


In [13]:
df_customer = dynamicFrameCustomers.toDF()
df_customer.show(3)

+----------+---------+-----------+----------------+
|customerid|firstname|   lastname|        fullname|
+----------+---------+-----------+----------------+
|       293|Catherine|       Abel|  Catherine Abel|
|       295|      Kim|Abercrombie| Kim Abercrombie|
|       297| Humberto|    Acevedo|Humberto Acevedo|
+----------+---------+-----------+----------------+
only showing top 3 rows


#### Example: Some DF processing


In [14]:
# Selecting certain fields from a Dynamic DataFrame
df_select = df_customer.select(["customerid", "fullname"])

#Drop Fields of Dynamic Frame
#df_customer = df_customer.drop(["firstname","lastname"])

# Show top 10
df_select.show(2)

+----------+---------------+
|customerid|       fullname|
+----------+---------------+
|       293| Catherine Abel|
|       295|Kim Abercrombie|
+----------+---------------+
only showing top 2 rows


#### Example: Add Columns In A Spark Dataframe

In [15]:
#import lit from sql functions 
from  pyspark.sql.functions import lit

# Add new column to spark dataframe
dfNewColumn = df_customer.withColumn("date", lit("2022-07-24"))

# show df with new column
dfNewColumn.show(3)

+----------+---------+-----------+----------------+----------+
|customerid|firstname|   lastname|        fullname|      date|
+----------+---------+-----------+----------------+----------+
|       293|Catherine|       Abel|  Catherine Abel|2022-07-24|
|       295|      Kim|Abercrombie| Kim Abercrombie|2022-07-24|
|       297| Humberto|    Acevedo|Humberto Acevedo|2022-07-24|
+----------+---------+-----------+----------------+----------+
only showing top 3 rows


#### Example: Using concat to concatenate two columns together

In [17]:
#import concat from functions 
from  pyspark.sql.functions import concat

# create another full name column
dfNewFullName = df_customer.withColumn("new_full_name",concat("firstname",concat(lit(' '),"lastname")))

#show full name column 
dfNewFullName.show(5)

+----------+---------+-----------+----------------+----------------+
|customerid|firstname|   lastname|        fullname|   new_full_name|
+----------+---------+-----------+----------------+----------------+
|       293|Catherine|       Abel|  Catherine Abel|  Catherine Abel|
|       295|      Kim|Abercrombie| Kim Abercrombie| Kim Abercrombie|
|       297| Humberto|    Acevedo|Humberto Acevedo|Humberto Acevedo|
|       291|  Gustavo|     Achong|  Gustavo Achong|  Gustavo Achong|
|       299|    Pilar|   Ackerman|  Pilar Ackerman|  Pilar Ackerman|
+----------+---------+-----------+----------------+----------------+
only showing top 5 rows


#### Example: Renaming columns

In [19]:
# Rename column in Spark dataframe
dfRenameCol = df_customer.withColumnRenamed("fullname","full_name")

#show renamed column dataframe
dfRenameCol.show(5)

+----------+---------+-----------+----------------+
|customerid|firstname|   lastname|       full_name|
+----------+---------+-----------+----------------+
|       293|Catherine|       Abel|  Catherine Abel|
|       295|      Kim|Abercrombie| Kim Abercrombie|
|       297| Humberto|    Acevedo|Humberto Acevedo|
|       291|  Gustavo|     Achong|  Gustavo Achong|
|       299|    Pilar|   Ackerman|  Pilar Ackerman|
+----------+---------+-----------+----------------+
only showing top 5 rows


#### Example: GroupBy and Aggregate Operations

In [22]:
# Group by lastname then print counts of lastname and show
df_customer.groupBy("lastname").count().show(5)

+--------+-----+
|lastname|count|
+--------+-----+
|  Achong|    1|
|  Bailey|    1|
|   Caron|    1|
|   Casts|    1|
|   Curry|    1|
+--------+-----+
only showing top 5 rows


#### Example: Filtering Columns and Where clauses

In [24]:
# Filter spark DataFrame for customers who have the last name Adams
# Using Filter method
df_customer.filter(df_customer["lastname"] == "Adams").show()

# using where clause
# Where clause spark DataFrame for customers who have the last name Adams
df_customer.where("lastname =='Adams'").show()

+----------+---------+--------+-------------+
|customerid|firstname|lastname|     fullname|
+----------+---------+--------+-------------+
|       305|    Carla|   Adams|  Carla Adams|
|       301|  Frances|   Adams|Frances Adams|
|       307|      Jay|   Adams|    Jay Adams|
+----------+---------+--------+-------------+


#### Example: Joins

In [25]:
# Read up another datset ordersand convert to spark dataframe
# Read from the customers table in the glue data catalog using a dynamic frame and convert to spark dataframe
dfOrders = glueContext.create_dynamic_frame.from_catalog(
                                        database = "pyspark_tutorial_db", 
                                        table_name = "orders"
                                    ).toDF()

In [33]:
#### Example: Inner join for Spark Dataframe All Data
# Inner Join Customers Spark DF to Orders Spark DF
df_combine = df_customer.join(dfOrders,df_customer.customerid ==  dfOrders.customerid,"inner")
#left join on orders and adams df
# df_customer.join(dfOrders,df_customer.customerid ==  dfOrders.customerid,"left").show(5, truncate=False)
df_combine.show(5, truncate= False)

+----------+---------+--------+--------------+------------+------------------+---------+---------+--------+----------+----------+----------+---------+--------+----------+---------+--------+---------+-----------------+---------+
|customerid|firstname|lastname|fullname      |salesorderid|salesorderdetailid|orderdate|duedate  |shipdate|employeeid|customerid|subtotal  |taxamt   |freight |totaldue  |productid|orderqty|unitprice|unitpricediscount|linetotal|
+----------+---------+--------+--------------+------------+------------------+---------+---------+--------+----------+----------+----------+---------+--------+----------+---------+--------+---------+-----------------+---------+
|517       |Richard  |Bready  |Richard Bready|43665       |61                |5/31/2011|6/12/2011|6/7/2011|283       |517       |14352.7713|1375.9427|429.9821|16158.6961|711      |2       |20.1865  |0.0000           |40.3730  |
|517       |Richard  |Bready  |Richard Bready|43665       |62                |5/31/2011|

#### Example: Writing data down using the Glue Data Catalog

##### Note: convert from spark Dataframe to Glue Dynamic DataFrame

In [34]:
# Import Dynamic DataFrame class
from awsglue.dynamicframe import DynamicFrame

#Convert from Spark Data Frame to Glue Dynamic Frame
dyfcombined = DynamicFrame.fromDF(df_combine, glueContext, "convert")

#Show converted Glue Dynamic Frame
dyfcombined.show(5)

{"customerid": 295, "firstname": "Kim", "lastname": "Abercrombie", "fullname": "Kim Abercrombie", "salesorderid": 44110, "salesorderdetailid": 1732, "orderdate": "8/1/2011", "duedate": "8/13/2011", "shipdate": "8/8/2011", "employeeid": 277, "subtotal": 16667.3077, "taxamt": 1600.6864, "freight": 500.2145, "totaldue": 18768.2086, "productid": 765, "orderqty": 2, "unitprice": 419.4589, "unitpricediscount": 0.0000, "linetotal": 838.9178}
{"customerid": 295, "firstname": "Kim", "lastname": "Abercrombie", "fullname": "Kim Abercrombie", "salesorderid": 44110, "salesorderdetailid": 1733, "orderdate": "8/1/2011", "duedate": "8/13/2011", "shipdate": "8/8/2011", "employeeid": 277, "subtotal": 16667.3077, "taxamt": 1600.6864, "freight": 500.2145, "totaldue": 18768.2086, "productid": 729, "orderqty": 4, "unitprice": 183.9382, "unitpricediscount": 0.0000, "linetotal": 735.7528}
{"customerid": 295, "firstname": "Kim", "lastname": "Abercrombie", "fullname": "Kim Abercrombie", "salesorderid": 44110, "

## Example: Write output data to S3 location

#### Example: 1. Write Dynamic DataFrame down to S3 location

In [35]:
# write down the data in converted Dynamic Frame to S3 location. 
glueContext.write_dynamic_frame.from_options(
                            frame = dyfcombined,
                            connection_type="s3", 
                            connection_options = {"path": "s3://pyspark-with-glue-learning-20231202"}, 
                            format = "csv", 
                            format_options={
                                "separator": ","
                                },
    # transformation_ctx which is used to identify state information for a job bookmark.
                            transformation_ctx = "datasink2")

#### Example: 2. Write Dynamic DataFrame down to S3 location

In [36]:
# write data from spark dataframe to the table using the meta data stored in the glue data catalog 
glueContext.write_dynamic_frame.from_catalog(
    frame = dyfcombined,
    database = "pyspark_tutorial_db",  
    table_name = "customers_write_dyf")

#### Example: Write the data in the DynamicFrame to a location in Amazon S3 and a table for it in the AWS Glue Data Catalog


In [ ]:
s3output = glueContext.getSink(
  path="s3://bucket_name/folder_name",
  connection_type="s3",
  updateBehavior="UPDATE_IN_DATABASE",
  partitionKeys=[],
  compression="snappy",
  enableUpdateCatalog=True,
  transformation_ctx="s3output",
)
s3output.setCatalogInfo(
  catalogDatabase="demo", catalogTableName="populations"
)
s3output.setFormat("glueparquet")
s3output.writeFrame(DyF)